## Chapter 02. GPT

#### Colab과 Google Drive 연동

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

#### KoGPT2

In [ ]:
# !pip install transformers

In [1]:
from transformers import GPT2LMHeadModel, TFGPT2LMHeadModel, PreTrainedTokenizerFast
# GPT2LMHeadModel = GPT2를 이용해 다음 단어를 예측하는 언어 모델
# TFGPT2LMHeadModel = Tensorflow용 버전, 위는 PyTorch용 버전; 사용하는 라이브러리에 맞게 하나만 사용
# PreTrainedTokenizerFast = 토크나이저

In [2]:
# PyTorch 모형 로딩
model = GPT2LMHeadModel.from_pretrained("taeminlee/kogpt2") # 모델 이름 지정
# PC는 지정 폴더 저장이라, 한 번 저장 시 재다운로드 X, Colab은 Google Drive 연동을 통해 필요할 때 호출

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

c:\Users\lucky\anaconda3\envs\TEST\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lucky\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [3]:
# TensorFlow 모형 로딩. 원본 모형이 PyTorch 용이므로 form_pt=True 로 설정
model = TFGPT2LMHeadModel.from_pretrained("taeminlee/kogpt2", from_pt=True)
# model card에서 전용 라이브러리를 확인할 수 있음

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'lm_head.weight', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [4]:
# 토크나이저 로딩
tokenizer = PreTrainedTokenizerFast.from_pretrained("taeminlee/kogpt2")

tokenizer_config.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


#### Google Drive에 모형 저장 및 불러오기

In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# git을 이용해 모형 복제
!sudo apt install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2 /gdrive/MyDrive/kogpt2/taeminlee

In [ ]:
# 모형 및 토크나이저 로딩
model = GPT2LMHeadModel.from_pretrained("/gdrive/MyDrive/kogpt2/taeminlee")

In [ ]:
model = TFGPT2LMHeadModel.from_pretrained("/gdrive/MyDrive/kogpt2/taeminlee", from_pt=True)

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("/gdrive/MyDrive/kogpt2/taeminlee")

#### 문장 생성

In [5]:
# Tensor Flow 모형 사용 시, return_tensors='pt'
# PyTorch 모형 사용 시, return_tensors='pt'
input_ids = tokenizer.encode('자연어 처리를', add_special_tokens=False, 
                             return_tensors="tf")
# 특수 토큰은 붙히지 않음 (add_special_tokens=False)

In [6]:
result = model.generate(
    input_ids,
    max_length=50, 
    num_beams=5, 
    early_stopping=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id)
# eos_token = 문장의 끝을 나타내는 토큰
# pad_token = 여러 문장의 길이를 맞출 때, 빈칸 대용으로 사용하는 토큰

In [7]:
tokenizer.decode(result[0])

'자연어 처리를 할 수 있다.</s><pad><pad><pad><pad>'

#### 자동 미분

In [8]:
# 자동 미분을 통해 경사하강법을 직접 적용하는 방법
# 딥러닝은 경사하강법을 통해 업데이트를 진행하는데, 함수의 기울어진 정도(경사)를 구하기 위해 미분을 진행한다
import tensorflow as tf

x = tf.Variable(3.0)
# tensor의 특별한 형태 Variable, tensor flow 내부에서 변수에 해당하는 형태
# x 라는 변수는 3.0이라는 값을 갖는다
x

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.0>

In [9]:
with tf.GradientTape() as tape:
    y = x ** 2
# y 는 x의 제곱이 되며, 자동미분을 위해 GradientTape 구문 안에서 진행한다
# with 는 특정 처리를 특정 맥락에서 진행하기 위해 사용한다
# 계산기록을 tape라는 변수에 저장

In [10]:
y

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [ ]:
# y를 x로 미분
# x가 변화할 때, y가 얼마나 변화하는가? 순간적인 기울기 계산
grad = tape.gradient(y, x)

In [16]:
grad

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [15]:
# 계산 결과를, 텐서 플로우 혹은 딥 러닝, 머신 러닝 모형을 만들었을 때 경사를 적용해줘야 함
# 오차 혹은 손실을 계산하여 최소화하는 시점을 찾아야 함
# y를 가장 작게 만들어주는 지점을 찾아주는 경사하강법 알고리즘 (Adam, SGD 등)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

In [17]:
# 기존에 구한 경사를 적용, 리스트를 만들어서 함수에 넣어줌
# 기존에 구한 경사를 x에 적용해 x를 업데이트
optimizer.apply_gradients([(grad, x)])

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [18]:
# 3.0 에서 2.9로 줄어듦
x

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.9000008>

In [19]:
with tf.GradientTape() as tape:
    y = x ** 2

In [20]:
# 9.0 에서 8.41로 줄어듦
# y가 줄어드는 방향으로 x를 변화시킨 것
y

<tf.Tensor: shape=(), dtype=float32, numpy=8.410005>

In [21]:
# old_y = 100 # 큰 값으로 지정
# new_y = 10
# while abs(old_y - new_y) > 0.1:
#     with tf.GradientTape() as tape:
#         y = x ** 2
#     grad = tape.gradient(y, x)
#     optimizer.apply_gradients([(grad, x)])
# 위와 같은 방식으로 코딩 가능

for i in range(30):
    with tf.GradientTape() as tape:
        y = x ** 2
    grad = tape.gradient(y, x)
    optimizer.apply_gradients([(grad, x)])
    print(x.numpy(), y.numpy())
# 같은 방식으로 지속적으로 x를 바꿔나가면서 y가 최소화 될 때까지 반복문 진행
# y 계산 → 경사 구하기 → 경사를 x에 적용
# .numpy를 통해 결과값을 숫자값만 표시
# 지속적인 반복을 통해 최솟점 근처로 가게 되면, y가 상승할 수도 있음
# y에 실제 딥러닝 모델을 집어넣게 되면 학습 진행 가능

2.8001046 8.410005
2.700384 7.8405857
2.6009169 7.2920732
2.5017836 6.7647686
2.4030683 6.258921
2.3048592 5.7747374
2.2072473 5.3123755
2.110327 4.8719406
2.014196 4.4534802
1.9189541 4.0569854
1.8247049 3.682385
1.7315532 3.329548
1.6396064 2.9982765
1.5489731 2.688309
1.4597634 2.3993175
1.372088 2.1309092
1.2860576 1.8826253
1.2017828 1.6539441
1.1193731 1.4442819
1.0389359 1.2529961
0.9605769 1.0793878
0.88439816 0.922708
0.8104983 0.7821601
0.7389711 0.6569075
0.669905 0.5460783
0.6033824 0.44877273
0.53947866 0.36407033
0.47826162 0.29103723
0.41979086 0.22873418
0.36411703 0.17622437


#### Tensorflow Dataset

In [22]:
# tensor flow의 Data set 관리 기능
# Data를 갖고 여러가지 처리를 할 수 있는 기능
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6, 7, 8])
# Dataset 이후 함수로 여러 방식으로 만들 수 있음
# 기존 tensor의 8건의 데이터를 하나의 Data set으로 만듦

In [23]:
# for문을 통해 각 값들을 하나 하나의 batch로 처리 가능
for batch in dataset:
    print(batch)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


In [24]:
# batch를 2개씩 묶어서 처리
for batch in dataset.batch(2):
    print(batch)

tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([3 4], shape=(2,), dtype=int32)
tf.Tensor([5 6], shape=(2,), dtype=int32)
tf.Tensor([7 8], shape=(2,), dtype=int32)


In [25]:
# batch를 4개씩 묶어서 처리
for batch in dataset.batch(4):
    print(batch)

tf.Tensor([1 2 3 4], shape=(4,), dtype=int32)
tf.Tensor([5 6 7 8], shape=(4,), dtype=int32)


In [26]:
# Data set을 list 등의 고정된 데이터 셋이 아니라 generator로 저장 가능
# for i in range(10): 이라는 함수를 실행할 때, 0부터 9까지 숫자들의 sequence를 만들어주는 역할을 함
# 이와 range(10)과 list(range(10))의 차이가 generator의 역할; 데이터를 생성할 수 있는 기능만 만들어 놓은 뒤, 필요할 때마다 데이터를 생성하는 방식
# return 은 함수의 결과값을 반환한 뒤 함수를 종료하지만, yield는 list의 경우, for 문을 돌면서 list의 원소를 하나씩 생성하게 됨
# gen() 은 실제 데이터를 갖고 있지 않지만 list(gen())을 실행 시, 데이터 생성
def gen():
    for i in range(10):
        yield [i] * i

In [27]:
# generator에서 만드는 Dataset
# output_type을 통해 자료형을 지정; generator는 아직 데이터를 생성한 결과가 아니기 때문에, tensor_slice와 달리 이미 데이터가 있는 걸 Data set으로 만드는 것이 아니다
dataset = tf.data.Dataset.from_generator(gen, output_types=tf.int32)

In [28]:
# padded_batch를 통해, 데이터를 묶을 때 패딩을 진행
# 하나의 배치 안에서 문장의 길이가 서로 다른 데이터를 일정하게 처리
# None 으로 지정 안할 시, 가장 긴 것에 맞춰서 길이를 맞춰줌 (예; 3의 길이에 맞춰 1, 2에 0이 추가되어 길이가 맞춰짐)
for batch in dataset.padded_batch(4, padded_shapes=(None, )):
    print(batch)

tf.Tensor(
[[0 0 0]
 [1 0 0]
 [2 2 0]
 [3 3 3]], shape=(4, 3), dtype=int32)
tf.Tensor(
[[4 4 4 4 0 0 0]
 [5 5 5 5 5 0 0]
 [6 6 6 6 6 6 0]
 [7 7 7 7 7 7 7]], shape=(4, 7), dtype=int32)
tf.Tensor(
[[8 8 8 8 8 8 8 8 0]
 [9 9 9 9 9 9 9 9 9]], shape=(2, 9), dtype=int32)


#### 챗봇 전처리

In [ ]:
# !pip install transformers
# !wget -c https://github.com/songys/Chatbot_data/raw/master/ChatbotData%20.csv

In [30]:
import pandas as pd
df = pd.read_csv('./data/ChatbotData.csv')
df.head()
# 사용자 입력에 대한 챗봇의 답변 데이터

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [31]:
# 전처리 전에 토큰화를 해야되므로 토크나이저 호출
from transformers import TFGPT2LMHeadModel, PreTrainedTokenizerFast

In [ ]:
# 사전학습 모델 입력
# pytorch 모델로 되어있을 경우, from_pt=True 옵션 지정
model = TFGPT2LMHeadModel.from_pretrained("taeminlee/kogpt2", from_pt=True)
tokenizer = PreTrainedTokenizerFast.from_pretrained("taeminlee/kogpt2")

In [33]:
# 표를 한 행씩 보기; itertuple
# f 문자열을 활용해, Question과 Answer를 연결해 하나의 문장으로 만듦
# SKT에서 원래 만든 토크나이저엔 문제가 없었지만, 현재 토크나이저는 tokenizer.vocab_size가 50,000지만 .vocab하면 124개가 더 많다
# 124개의 토큰이 토크나이저에 잘못 들어가 있기 때문에 for 문에 있는 t를 t로서 list에 넣는 식을 활용해 50,000개의 토큰만 활용
data = []
for row in df.itertuples():
    tokens = tokenizer.encode(f'<s>Q: {row.Q} A: {row.A}</s>')
    tokens = [t for t in tokens if t < tokenizer.vocab_size]
    data.append(tokens)

In [34]:
len(data)

11823

In [35]:
data[0] # 토큰 번호
# 문장의 시작과 끝을 나타내는 토큰을 넣기 위해 위의 순환문 중 <s>와 </s> 를 추가함
tokenizer.bos_token # 문장의 시작을 나타내는 토큰
tokenizer.eos_token # 문장의 끝을 나타내는 토큰

'</s>'

#### 챗봇 Dataset

In [36]:
# 만든 데이터를 tensor flow에서 다룰 수 있게 처리
import tensorflow as tf

# 변환을 위해 함수를 만듦
def data_generator():
    for datum in data:
        yield datum
        
# Data를 필요할 때마다 가져와 tensor flow에 Data set 형태로 덮어줌
dataset = tf.data.Dataset.from_generator(data_generator, output_types=tf.int32)

# 딥러닝의 학습은 여러 개의 데이터를 묶은 하나의 배치 단위로 이뤄지기 때문에 배치로 바꿔줘야함
# 각각의 대화가 길이가 다르기 때문에 padded_batch를 통해 길이를 맞춰줘야 함
# 토크나이저의 패딩용 토큰을 채워넣을 값을 지정
dataset = dataset.padded_batch(32, padded_shapes=(None, ), padding_values=tokenizer.pad_token_id)

# 하나의 배치를 꺼내볼 수 있음
# 32개의 데이터가 묶여 있으며, 가장 긴 길이의 데이터를 기준으로 28의 길이로 맞춰져 있음
# 모자란 토큰은 3번 토큰으로 채워져 있으며, 패딩용 토큰이다
# 이후 batch를 하나씩 꺼내 모형에 넣어 학습을 진행
batch = next(iter(dataset))
batch

<tf.Tensor: shape=(32, 28), dtype=int32, numpy=
array([[    0,  4322, 47820,   385, 47460, 47437, 49108, 47812,   450,
        47820, 33203,   252,   119,  7974, 47440,     1,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,
            3],
       [    0,  4322, 47820,   106, 47445, 47766,  1084,  1024, 47816,
        47487,   450, 47820,  5886, 47466, 19188, 47440,     1,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,
            3],
       [    0,  4322, 47820,   141, 47650, 47514, 47471,  2211, 47593,
         2999,  5314,   450, 47820,  1791, 47459, 15515,   548, 48159,
        47440,     1,     3,     3,     3,     3,     3,     3,     3,
            3],
       [    0,  4322, 47820,   141, 47650, 47514, 47471,  1057,  2211,
        47593,  2999,  5314,   450, 47820,  1791, 47459, 15515,   548,
        48159, 47440,     1,     3,     3,     3,     3,     3,     3,
            3],
       [    0,  4322, 47820,   753, 

#### 챗봇 데이터로 미세 조정

In [37]:
# 미세조정을 위한 optimizer 알고리즘 초기화
# 세분적인 방법에 따라 여러 알고리즘 선택 가능 (Adam을 가장 많이 사용)
# 한 번에 파라미터를 얼마나 업데이트 할 것이냐? 클수록 학습은 빠르지만 학습이 불안정해짐 (learning_rate) ; 3e-5 = 3 x 10의 -5승
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [38]:
result = model(batch, labels=batch) # 정답을 batch를 통해 자기자신으로 주어져야 예측만 하지 않고 손실 값을 구해줌
result['loss'] # 손실값은 문장의 시작을 나타내는 토큰 이후 부터 예측을 시작해 Q라는 단어가 나올 확률(교차 엔트로피 값)부터 이어짐
# padding으로 되어있는 부분은 손실을 계산하지 않기 때문에 0으로 계산해 loss에 반영하지 않음

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.824298], dtype=float32)>

In [39]:
tf.reduce_mean(result['loss']) # 평균적인 손실을 낮춰 다음에 나온 단어를 높은 확률로 예측할 수 있게 만듦

<tf.Tensor: shape=(), dtype=float32, numpy=4.824298>

In [40]:
# 전체 길이를 batch 사이즈로 구한 뒤 소숫점 뒤를 버리고 총 batch 개수는 하나 더 더해서 구함
total = df.shape[0] // 32 + 1

In [41]:
# 경사하강법을 통해 mean_loss를 y 값을 지정해 평균 손실이 최소화되는 지점을 찾음
# trainable_variables을 조정해 loss를 낮추는 것을 찾음
# apply_gradient를 통해 loss를 적용해 기존 파라미터를 loss가 줄어드는 방향으로 수정함

with tf.GradientTape() as tape:
    result = model(batch, labels=batch)
    loss = result['loss']
    mean_loss = tf.reduce_mean(loss)

grads = tape.gradient(mean_loss, model.trainable_variables)
opt.apply_gradients(zip(grads, model.trainable_variables))
mean_loss.numpy()

4.824298

In [42]:
import tqdm.notebook

# 위의 코드를 서로 다른 batch에 대해 반복
# 한 번의 순환이 한 epoche 이며, 미세 조정은 너무 많은 epoche를 진행할 경우, 기존 학습을 망각할 수 있기 때문에 좋지 않을 수 있음
for batch in tqdm.notebook.tqdm(dataset, total=total):
    with tf.GradientTape() as tape:
        result = model(batch, labels=batch)
        loss = result['loss']
        mean_loss = tf.reduce_mean(loss)

    grads = tape.gradient(mean_loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables))

  0%|          | 0/370 [00:00<?, ?it/s]

In [43]:
# 추후 활용을 위해 모델 저장
model.save_pretrained("./model/chat_model")

#### 챗봇과 대화하기

In [44]:
# 시작 토큰을 입력 후 Q 를 입력하고 A 이후로 챗봇의 말을 만듦
text = '오늘도 좋은 하루!'
sent = f'<s>Q: {text} A:'

In [45]:
sent

'<s>Q: 오늘도 좋은 하루! A:'

In [46]:
# 토크나이저에 인코딩하며, 전처리할 때와 달리 바로 tensor flow 형식으로 변환
input_ids = tokenizer.encode(sent, return_tensors='tf')

In [47]:
# 모델에 입력
# 최대 길이 50 토큰으로 지정
# early_stopping = Ture 를 통해, 문장의 끝을 나타내는 토큰을 만났을 시 종료
# eos token 은 토크나이져에 지정되어있는 토큰 지정
# pad_token 은 필수는 아니지만 토크나이져의 패드 토큰 지정
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id)

c:\Users\lucky\anaconda3\envs\TEST\lib\site-packages\transformers\generation\configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [48]:
tokenizer.decode(output[0])

'<s> Q: 오늘도 좋은 하루! A: 좋은 하루 되세요.</s>'

In [49]:
answer = tokenizer.decode(output[0])
# 입력된 문장의 길이가 sent의 길이이므로
answer[len(sent):-len('</s>')]
# 이와 같이 대답 출력

': 좋은 하루 되세요.'

In [50]:
# 랜덤하게 답변을 생성
output = model.generate(input_ids, max_length=50, do_sample=True, early_stopping=True, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id)
tokenizer.decode(output[0])

'<s> Q: 오늘도 좋은 하루! A: 좋은 일이 있을 거예요.</s>'

### BLEU

In [53]:
!pip show nltk
# 설치된 nltk의 버젼을 확인 (현재 3.2.5)
# 낮은 버젼에서 BLEU 계산에 문제가 생김 (3.5 이상이여야 함)

Name: nltk
Version: 3.8.1
Summary: Natural Language Toolkit
Home-page: https://www.nltk.org/
Author: NLTK Team
Author-email: nltk.team@gmail.com
License: Apache License, Version 2.0
Location: c:\users\lucky\anaconda3\envs\test\lib\site-packages
Requires: click, joblib, regex, tqdm
Required-by: 


In [ ]:
# !pip install -U nltk
# nltk를 업데이트하여 BLEU 계산 문제 해결 (3.2.5를 제거하고 3.5 이상 설치)

In [54]:
# nltk의 BLEU 계산 함수 모듈 호출
from nltk.translate.bleu_score import SmoothingFunction, modified_precision, sentence_bleu

# 기준 문장과 생성했다고 가정할 문장
# .split을 통해 빈 칸 단위로 끊어 토큰화
# BLEU는 정답이 여러 개라면, 여러 개를 갖고 계산할 수 있으므로, 기준 문장을 리스트에 담아 변수 지정
ref = ['the cat is on the mat'.split()]
sentence = 'the the the the the the the'.split()

In [55]:
# MUP 계산, 몇 개의 단위의 토큰으로 비교할 것인가 3번째 인자로 지정 (현재, unigram)
p1 = modified_precision(ref, sentence, 1)
float(p1)

0.2857142857142857

In [56]:
# 2-gram preicision (bi-gram)
p2 = modified_precision(ref, sentence, 2)
float(p2)

0.0

In [57]:
# N이 1부터 4까지, 기하평균을 구한 뒤 패널티를 포함해 BLEU 점수 계산
# 0점이 들어가기 때문에 아주 작은 수를 도출함(smoothing method_1)
sentence_bleu(ref, sentence)

c:\Users\lucky\anaconda3\envs\TEST\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\lucky\anaconda3\envs\TEST\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\lucky\anaconda3\envs\TEST\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or us

1.331960397810445e-231

In [58]:
# smoothing function 지정을 통해 다른 방식으로 BLEU 계산
# method2 = 분모와 분자에 각각 1씩 더해주는 방식
sentence_bleu(ref, sentence, smoothing_function=SmoothingFunction().method2)

0.19205612637498934

### TFRecord

In [59]:
# tensor flow에서 사용하는 데이터 저장 방식
# 너무 큰 데이터인 경우, 한 번에 불러올 경우 메모리를 과도하게 차지하기 때문에, 나눠서 불러오는데, 기존 csv 등의 포멧은 데이터를 신속하게 불러올 수 없음
# 데이터 하나를 동시에 읽을 수도 있어야 하기 때문에 tensor flow에서 TFRecord 라는 용량도 작으며, 빠른 속도로 불러올 수 있는 데이터 형식 제공
# TFRecord는 일반적으로 example(=row)로 저장하며, Feature를 통해 columns을 형성한다

import tensorflow as tf

def bytes_feature(value):  # 문자열
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def float_feature(value):  # 실수
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def int_feature(value):  # 정수
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

# TFRecord에서 위 3가지 형태의 데이터만 제시함
# 다른 자료형의 경우, 위 3가지 형태로 변환시켜 줘야함

In [60]:
example = tf.train.Example(features=tf.train.Features(feature={
    'name': bytes_feature(['김갑'.encode('utf8')]),
    'age': int_feature([16]),
    'height': float_feature([174.1]),
    'item': int_feature([1, 2, 3])
}))
# {} 내부에 사전형태로 정보 지정
# Feature 들은 무조건 리스트 형태로 지정되어야 하며, 문자열은 인코딩을 포함해야 함
example.SerializeToString()
# 데이터 전체를 압축된 형태로 표현 (컴퓨터가 읽고 쓰기 쉬운 형태)

b'\nG\n\x12\n\x04name\x12\n\n\x08\n\x06\xea\xb9\x80\xea\xb0\x91\n\x0f\n\x04item\x12\x07\x1a\x05\n\x03\x01\x02\x03\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01\x10\n\x12\n\x06height\x12\x08\x12\x06\n\x04\x9a\x19.C'

#### TFRecord 쓰기

In [61]:
def serialize_example(name, age, height, item):
    example = tf.train.Example(features=tf.train.Features(feature={
        'name': bytes_feature([name.encode('utf8')]),
        'age': int_feature([age]),
        'height': float_feature([height]),
        'item': int_feature(item)
    }))
    return example.SerializeToString()
# 위의 과정을 함수로 설정

In [62]:
filename = 'test1.tfrecord'
with tf.io.TFRecordWriter(filename) as writer:
    s = serialize_example('김갑', 16, 174.1, [1, 2, 3])
    writer.write(s)

    s = serialize_example('이을', 35, 182.6, [1, 2])
    writer.write(s)

In [63]:
filename = 'test2.tfrecord'
with tf.io.TFRecordWriter(filename) as writer:
    s = serialize_example('박병', 24, 169.5, [2])
    writer.write(s)

#### TFRecord 읽기

In [64]:
filenames = ['test1.tfrecord', 'test2.tfrecord']
dataset = tf.data.TFRecordDataset(filenames)
# file을 리스트 형태로 이름을 써준 뒤, tensorflow의 TFRecordDataset에 데이터 이름을 넣어주면, 파일이 몇 개든 하나의 Dataset으로 취급해줌

In [65]:
# Dataset 사용법
# 하나의 example 씩 읽어주게 됨
for batch in dataset:
    example = tf.train.Example() # 빈 Example을 생성
    example.ParseFromString(batch.numpy()) # batch에서 Example을 읽어옴
    print(example)

features {
  feature {
    key: "name"
    value {
      bytes_list {
        value: "\352\271\200\352\260\221"
      }
    }
  }
  feature {
    key: "item"
    value {
      int64_list {
        value: 1
        value: 2
        value: 3
      }
    }
  }
  feature {
    key: "height"
    value {
      float_list {
        value: 174.1
      }
    }
  }
  feature {
    key: "age"
    value {
      int64_list {
        value: 16
      }
    }
  }
}

features {
  feature {
    key: "name"
    value {
      bytes_list {
        value: "\354\235\264\354\235\204"
      }
    }
  }
  feature {
    key: "item"
    value {
      int64_list {
        value: 1
        value: 2
      }
    }
  }
  feature {
    key: "height"
    value {
      float_list {
        value: 182.6
      }
    }
  }
  feature {
    key: "age"
    value {
      int64_list {
        value: 35
      }
    }
  }
}

features {
  feature {
    key: "name"
    value {
      bytes_list {
        value: "\353\260\225\353\263\

In [66]:
# .features.feature['age'] 를 통해 자료를 확인할 수 있고, 해당 자료 출력 형태를 따라가면 최종적인 값만 출력 가능
example.features.feature['age'].int64_list.value[0]

24

In [67]:
example.features.feature['name'].bytes_list.value[0].decode('utf8')

'박병'

In [68]:
example.features.feature['height'].float_list.value[0]

169.5

In [69]:
example.features.feature['item'].int64_list.value

[2]

#### Dataset을 TFRecord로 쓰기

In [70]:
# 데이터를 하나씩 저장, 호출하는 것이 아닌, 한꺼번에 저장, 호출하는 방식
import pandas as pd

# 데이터 프레임 형성
df = pd.DataFrame({
    'name': ['김갑', '이을', '박병'],
    'age': [16, 35, 24],
    'height': [174.1, 182.6, 169.5],
    'items': [[1,2,3], [1,2], [2]]
})

# 한 행씩 example로 만들어서 문자열로 바꾸는 함수 생성 (위에서 생성한 함수 사용)
def generator():
    for row in df.itertuples():
        yield serialize_example(row.name, row.age, row.height, row.items)
        
recset = tf.data.Dataset.from_generator(generator, output_types=tf.string) # output_types 지정

# Dataset을 저장할 땐 experimental.TFRecordWriter 을 사용 (하나씩 저장할 땐 io.TFRecordWirter 사용)
writer = tf.data.experimental.TFRecordWriter('test3.tfrecord') # 파일명 지정
writer.write(recset) # Dataset을 한 번에 지정해서, 한 번에 저장

Instructions for updating:
To write TFRecords to disk, use `tf.io.TFRecordWriter`. To save and load the contents of a dataset, use `tf.data.experimental.save` and `tf.data.experimental.load`


#### TFRecord 일괄 변환

In [71]:
# Dataset 단위로 통채로 불러오기
dataset = tf.data.TFRecordDataset(['test1.tfrecord', 'test2.tfrecord'])
# 기존에 진행했던 하나씩 불러오는 방식

In [72]:
# 파일에 들어있는 example 의 형태를 정의
# 길이가 정해져 있다면 FixedLenFeature, 아니라면 FixedSequenceFeature (예를 들어, 리스트 형태)
# shape = () 빈 괄호 지정, 기존에 byte 였다면 dtpye = tf.string
# int는 64, float은 32 bit
feature_description = {
    'name': tf.io.FixedLenFeature(shape=(), dtype=tf.string),
    'age': tf.io.FixedLenFeature(shape=(), dtype=tf.int64),
    'height': tf.io.FixedLenFeature(shape=(), dtype=tf.float32),
    'item': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
}

In [73]:
# 하나만 불러왔을 때
batch = next(iter(dataset))
batch

<tf.Tensor: shape=(), dtype=string, numpy=b'\nG\n\x12\n\x04name\x12\n\n\x08\n\x06\xea\xb9\x80\xea\xb0\x91\n\x0f\n\x04item\x12\x07\x1a\x05\n\x03\x01\x02\x03\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01\x10\n\x12\n\x06height\x12\x08\x12\x06\n\x04\x9a\x19.C'>

In [74]:
# map 함수는 아래 함수를 가져와, dataset에 있는 모든 사례에 아래 함수를 적용해줌
dataset = dataset.map(lambda x: tf.io.parse_single_example(x, feature_description))

In [75]:
batch = next(iter(dataset))
batch
# dictionary에 tensor 형태로 데이터가 입력되어 있음

{'age': <tf.Tensor: shape=(), dtype=int64, numpy=16>,
 'height': <tf.Tensor: shape=(), dtype=float32, numpy=174.1>,
 'item': <tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 2, 3], dtype=int64)>,
 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'\xea\xb9\x80\xea\xb0\x91'>}

#### 토큰화된 데이터를 TFRecord로 저장

In [ ]:
#!pip install transformers
#!wget -c https://github.com/e9t/nsmc/raw/master/ratings_train.txt

In [3]:
import pandas as pd
nsmc = pd.read_csv('./data/ratings_train.txt', sep='\t')
nsmc.head()
# 영화 리뷰 데이터

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
import tensorflow as tf
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("taeminlee/kogpt2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [78]:
nsmc.shape

(150000, 3)

In [79]:
from sklearn.model_selection import train_test_split

small = nsmc.iloc[:10000]

# train_test_split
idx_train, idx_test = train_test_split(small.index, test_size=0.2, random_state=42)

In [80]:
idx_train, idx_test = train_test_split(small.index, test_size=0.2, random_state=42)
def int_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [81]:
df = small.loc[idx_train]
row = next(df.itertuples())
row

Pandas(Index=9254, id=6213543, document='80년대 최고의 공포영화..', label=1)

In [82]:
tokens = tokenizer.encode(row.document)
tokens = [t for t in tokens if t < tokenizer.vocab_size]

In [83]:
# 문장의 길이를 알아야 마지막 토큰을 알 수 있음
n = len(tokens)

In [84]:
example =tf.train.Example(features=tf.train.Features(feature={
    'tokens':int_feature(tokens),
    'n': int_feature([n]),
    'label':int_feature([row.label])
}))

In [85]:
# train data 저장
filename = 'train_nsmc.tfrecord'
df = small.loc[idx_train]

with tf.io.TFRecordWriter(filename) as writer:
    for row in df.itertuples():
        tokens = tokenizer.encode(row.document)
        tokens = [t for t in tokens if t < tokenizer.vocab_size]
        n = len(tokens)
        example = tf.train.Example(features=tf.train.Features(feature={
            'tokens': int_feature(tokens),
            'n': int_feature([n]),
            'label': int_feature([row.label])
        }))

        s = example.SerializeToString()
        writer.write(s)

In [86]:
# test data 저장
filename = 'test_nsmc.tfrecord'
df = small.loc[idx_test]

with tf.io.TFRecordWriter(filename) as writer:
    for row in df.itertuples():
        tokens = tokenizer.encode(row.document)
        tokens = [t for t in tokens if t < tokenizer.vocab_size]
        n = len(tokens)
        example = tf.train.Example(features=tf.train.Features(feature={
            'tokens': int_feature(tokens),
            'n': int_feature([n]),
            'label': int_feature([row.label])
        }))

        s = example.SerializeToString()
        writer.write(s)

### Zero-shot 감성 분석

In [1]:
# 특별한 미세 조정 없이, 문장이 주어졌을 때, 문장 뒤 올 단어를 예측해서 문장 끝에 긍/부정을 예측
from transformers import TFGPT2LMHeadModel

model = TFGPT2LMHeadModel.from_pretrained("taeminlee/kogpt2", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [4]:
text = nsmc.document.iloc[0]
text
# 부정적인 문장

'아 더빙.. 진짜 짜증나네요 목소리'

In [7]:
input_ids = tokenizer.encode(text, return_tensors='tf')
# 문장을 토크나이저에 입력

In [8]:
input_ids.shape

TensorShape([1, 10])

In [9]:
result = model(input_ids)
result['logits']

<tf.Tensor: shape=(1, 10, 50000), dtype=float32, numpy=
array([[[  2.1121006 ,   7.5509257 ,  -8.365714  , ...,  -2.09952   ,
           0.9347404 ,  -0.232217  ],
        [  0.69625425,   9.109236  , -11.216259  , ...,  -4.9662447 ,
          -4.410963  ,  -1.8078165 ],
        [  1.9094037 ,   8.90713   , -12.061606  , ...,  -3.8685226 ,
          -3.4481714 ,  -3.701539  ],
        ...,
        [ -0.7769085 ,  12.743277  , -13.720602  , ...,  -4.797595  ,
          -0.4287318 ,  -2.1140919 ],
        [  3.6529002 ,  13.500687  , -12.352482  , ...,  -2.738575  ,
          -2.8930151 ,  -2.0195506 ],
        [ -0.5156875 ,  10.563933  , -11.987965  , ...,  -2.1986065 ,
          -3.1426308 ,  -2.0834565 ]]], dtype=float32)>

In [10]:
tokenizer.encode('좋다')
# '좋다'가 해당되는 위치

[3311]

In [11]:
tokenizer.encode('별로')
# '별로'가 해당되는 위치 (싫다는 2개 위치가 나오므로, 1개 위치가 출력되는 표현 선택)

[9321]

In [12]:
# 좌표를 넣어주면, 좌표에 해당하는 데이터를 추출해 새로운 tensor를 만들어줌
# 0번부터 9번까지 (10개) 범위와, 긍부정에 해당하는 토큰을 지정
logits = tf.gather_nd(result['logits'], [[0, 9, 9321], [0, 9, 3311]])
logits

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([5.3892064, 5.831721 ], dtype=float32)>

In [13]:
# logits 값을 확률로 바꿔줌
tf.nn.softmax(logits)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.391142, 0.608858], dtype=float32)>

#### Zero-shot 감성 분석 테스트

In [95]:
# 하나의 문장으론 작동 테스트에 부적합하므로, 여러 문장을 통해 정확도 평가 진행
test_dataset = tf.data.TFRecordDataset(['test_nsmc.tfrecord'])

In [21]:
# 모델에 넣기 적합한 형태로 지정
feature_description = {
    'tokens': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
    'n': tf.io.FixedLenFeature(shape=(), dtype=tf.int64),
    'label': tf.io.FixedLenFeature(shape=(), dtype=tf.int64),
}

In [97]:
batch = next(iter(test_dataset))

In [98]:
# example 하나 예시 생성
tf.io.parse_single_example(batch, feature_description)

{'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 'n': <tf.Tensor: shape=(), dtype=int64, numpy=24>,
 'tokens': <tf.Tensor: shape=(24,), dtype=int64, numpy=
 array([ 1069,  7651,  8000,  4722, 47541, 48397, 47437,  7625,   127,
         8462,   677, 47494, 10206, 47818, 47461, 47569,   246, 47442,
         3461,   409, 10360,  4104, 47440, 10258], dtype=int64)>}

In [19]:
# .cast = data의 type을 변경해줌
def parse_data(example):
    data = tf.io.parse_single_example(example, feature_description)
    data['tokens'] = tf.cast(data['tokens'], tf.int32)
    data['n'] = tf.cast(data['n'], tf.int32)
    data['label'] = tf.cast(data['label'], tf.int32)
    return data

In [100]:
# test data의 위의 함수를 일괄적용하며 패딩을 함께 진행
test_dataset = test_dataset.map(parse_data).padded_batch(32, padding_values=tokenizer.pad_token_id)

In [101]:
# 성능 테스트
import tqdm.notebook
n_total = 2000 # test data 전체 개수
n_correct = 0 # 몇 개 맞았는지 저장

for batch in tqdm.notebook.tqdm(test_dataset, total=2000 // 32 + 1): # 전체 갯수를 batch 수로 나눈 뒤 +1
    result = model(batch['tokens']) # 모델에 batch의 tokens 값을 입력
    indices = [] # 문장 길이를 표시하기 위한 빈 list
    for i, n in enumerate(batch['n']): # n은 문장마다 길이를 의미
        j = n - 1 # 마지막 토큰의 위치 (예) 길이가 24개의 토큰이라면, 마지막 토큰의 길이는 23이 됨
        indices.append([[i, j, 9321], [i, j, 3311]])
    logits = tf.gather_nd(result['logits'], indices)

    probs = tf.nn.softmax(logits) # 일괄 처리를 통해 확률로 변환
    good_probs = probs[:, 1] # 긍부정의 기본적인 확률 자체가 차이가 있으므로, 기준을 0.5로 잡으면 긍정이 유리하므로 긍정에 해당하는 확률열만 따로 추출
    is_good = tf.where(good_probs > 0.5, 1, 0) # 긍정의 확률이 0.5보다 높으면 긍정, 낮으면 부정으로 예측, 기준값 0.5는 변경 가능
    n_correct += tf.reduce_sum(tf.cast(is_good == batch['label'], tf.int32)).numpy() # is_good과 batch의 label과 비교해, True, False가 출력되고 자료형을 변경해 결과 도출

  0%|          | 0/63 [00:00<?, ?it/s]

In [102]:
# Zero-shot 을 통해 아무런 추가 조정 없이 결과 도출
n_correct / 2000

0.5835

### KoGPT2 감성 분석을 위한 미세 조정

In [14]:
# 감성 분석 시 긍부정으로 문장 분류
from transformers import TFGPT2ForSequenceClassification

# 모델 호출 후 사전학습 진행
model = TFGPT2ForSequenceClassification.from_pretrained("taeminlee/kogpt2", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2ForSequenceClassification: ['transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2ForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassificat

In [15]:
result = model(input_ids)

In [16]:
# (1, 10, 2)
# 긍부정을 예측하기 때문에 2, 언어모형은 50,000
result['logits']

<tf.Tensor: shape=(1, 10, 2), dtype=float32, numpy=
array([[[-0.06495173, -0.03280668],
        [ 0.23053408,  0.02685528],
        [ 0.2291107 , -0.18320113],
        [-0.11420687, -0.15777808],
        [ 0.2834201 ,  0.15727204],
        [-0.1615361 , -0.23664558],
        [ 0.01690693,  0.06465401],
        [ 0.03262112,  0.08841263],
        [ 0.13037543,  0.03150433],
        [ 0.19479348, -0.17799076]]], dtype=float32)>

In [22]:
# 학습을 위한 train dataset 형성
train_dataset = tf.data.TFRecordDataset(['train_nsmc.tfrecord'])
train_dataset = train_dataset.map(parse_data).padded_batch(32, padding_values=tokenizer.pad_token_id)

In [ ]:
import math
n_total = 8000 # data 전체 개수
n_correct = 0

opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08) # optimizer 정의
loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=True) # logit과 비교하기 위한 손실 함수 정의

for batch in tqdm.notebook.tqdm(train_dataset, total=math.ceil(8000 / 32)): # = (8000 // 32 + 1)
    with tf.GradientTape() as tape: # logit 도출 부분 경사 계산
        result = model(batch['tokens'])
        indices = []
        for i, n in enumerate(batch['n']):
            j = n - 1
            indices.append([[i, j, 0], [i, j, 1]]) # 긍부정을 예측하기 때문에 0과 1만 가져오면 됨
        logits = tf.gather_nd(result['logits'], indices)
        loss = loss_func(batch['label'], logits[:, 1]) # logit과 loss를 바로 비교 (확률 도출 x) # logit과 batch의 실제 label과 비교
        # BinaryCrossEntropy기 때문에 긍정 부분만 비교하면 됨
    
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables)) # 경사를 모델에 적용

In [ ]:
# 기존에 만들어둔 test 코드
n_total = 2000
n_correct = 0

for batch in tqdm.notebook.tqdm(test_dataset, total=math.ceil(2000 / 32)):
    result = model(batch['tokens'])
    indices = []
    for i, n in enumerate(batch['n']):
        j = n - 1
        indices.append([[i, j, 0], [i, j, 1]])
    logits = tf.gather_nd(result['logits'], indices)

    probs = tf.nn.softmax(logits)
    good_probs = probs[:, 1]
    is_good = tf.where(good_probs > 0.5, 1, 0)
    n_correct += tf.reduce_sum(tf.cast(is_good == batch['label'], tf.int32)).numpy()

In [26]:
n_correct / 2000

0.847